In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import xarray as xr
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-23 12:21:08.880733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
x=12+3
list(range(3, x, 3))

[3, 6, 9, 12]

In [2]:
# TODO: put data in a data folder
df_olr = xr.load_dataset('olr_data.nc') 
df_wg = xr.load_dataset('wind_geopot_data.nc')

### Taking out 1982/83 season

In [3]:
#take out 1982-5 -> 1983-5
cut_time_slice1 = slice('1979', '1982-5')
cut_time_slice2 = slice('1983-11', '2019')

df_olr = xr.merge([df_olr.sel(time=cut_time_slice1), df_olr.sel(time=cut_time_slice2)])
df_wg = xr.merge([df_wg.sel(time=cut_time_slice1), df_wg.sel(time=cut_time_slice2)])


### Picking out the Anomolies for OLR Data

In [4]:
#calculate olr anomolies

#take the 21-day running means (for olr by mean)
olr_short = df_olr.olr.values
olr_byyear = np.reshape(olr_short, (212, 39, 13, 144))

season_meanolr = np.zeros((212, 13, 144))

for i in range(1,212):
    season_meanolr[i, :,:] = np.mean(np.squeeze(olr_byyear[i,:,:,:]))

season_meanolr

olr_day = np.roll(season_meanolr, (21,1))


olr_season = np.zeros((212, 39, 13, 144))

#subtract out the 21-day running mean to get anomalies
for i in range(1,39):
    for j in range(1,212):
        olr_season[j, i, :, :] = np.squeeze(olr_byyear[j,i,:,:]) - np.squeeze(olr_day[j,:,:])
olr_season


#take 5-day running mean to smooth out the data
olr = olr_season
olr = np.zeros((212, 39, 13, 144))

for i in range(1,39):
    olr[:, i, :, :] = np.roll(np.squeeze(olr_season[:, i, :, :]), (5,1))


df_olr.olr.values = np.reshape(olr, (8268, 13, 144))

### Post-Normalized Graphs of the Data

### Normalizing the Data

In [5]:
#normalizing z500 data
z500_means = np.mean(df_wg.z500.values)
z500_std = np.std(df_wg.z500.values)
df_wg.z500.values = (df_wg.z500.values - z500_means)/z500_std

#normalizing u250 data
u250_means = np.mean(df_wg.u250.values)
u250_std = np.std(df_wg.u250.values)
df_wg.u250.values = (df_wg.u250.values - u250_means)/u250_std

#normalizing olr data
olr_means = np.mean(df_olr.olr.values)
olr_std = np.std(df_olr.olr.values)
df_olr.olr.values = (df_olr.olr.values - olr_means)/olr_std


In [6]:
#creating a list of all the dates being used
label_list = list(np.datetime_as_string(df_olr.time.values))
for i, val in enumerate(label_list):
    label_list[i] = val.split('T')[0]

### Seperating PPH Data by Location and Type
This is where I made the daily means for pph values at each location. It took forever to run so I commented it out and downloaded it as a csv file called ('pph_loc_mean.csv'). 

In [7]:
pph_data = pd.read_csv('pph_loc_mean.csv').rename(columns={'Unnamed: 0': 'date'}).set_index('date')
pph_data

,west_tor,west_wind,west_hail,central_tor,central_wind,central_hail,east_tor,east_wind,east_hail
date,,,,,,,,,
1979-11-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1979-11-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1979-11-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1979-11-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1979-11-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2019-05-27,0.200550,0.038387,0.274401,3.712219,4.844967,5.126097,0.0,0.000000,0.000000
2019-05-28,0.012979,0.002045,0.064389,1.073510,3.527014,5.957370,0.0,0.587768,0.000000
2019-05-29,0.000000,0.000008,0.000890,2.231628,5.683658,2.266128,0.0,0.000000,0.000000


### Formatting data to output time-series-like

In [8]:
#creating windows and horizons
HORIZON = 7 # PPH from 7 days afterwords
WINDOW = 30 # assumes sliding in days


# creates a expanding window from given label -- WORKS!
def get_window_expanding(label="", label_list=label_list):
    try:
        # take the season
        # season_index = label_list.index(label)

        # grab all the days previous to one that you have
        window = [0]*(len(label_list)-7)
        for i in range(len(window)):
            window[i] = label_list[len(label_list)-i-8]
        # return window and label
        return window, label
        
    
    #printing error when value is not in seasons list
    except:
        return "Label not found in list"
    

# creates a sliding window from given label -- WORKS!
def get_window_sliding (label="", window_num=WINDOW):
    try: 
        #ensure not grabbing from diff season
        day_index = label_list.index(label)
        m = int(label.split('-')[1])
        d = int(label.split('-')[2])

        #for values that are almost outside the season, use and expanding window 
        if (m ==11 ) or (m ==12 and d < 7):
            y = label.split('-')[0]
            stopper = y+'-11-01' #end of the season
            stop_index = label_list.index(stopper)

            #creates a new list to grab data from
            label_list_copy = label_list.copy()[stop_index:day_index+1]

            return (get_window_expanding(label=label, label_list=label_list_copy))

        else:
            window = [0]*(window_num)

            for i in range(len(window)):
                window[i] = label_list[day_index-i-7] #TODO: 7 days between or pph_day - last_day = 7
            return window, label  
    
    except:
        return "Label not found in list"
    

# returns wind and geopotential dataframe for prev 30 days (minus 7d), olr dataframe for prev 30 days(minus 7d), 
# and pph data for the given date
def get_data(date):
    dates = get_window_sliding(date)
    past_dates = dates[0]
    target_data = pph_data.loc[date]
    # print(len(past_dates))
    if len(past_dates) == 0:
        return [], [], target_data
    slicer = slice(past_dates[-1], past_dates[0])

    return df_wg.sel(time=slicer), df_olr.sel(time=slicer), target_data

### Spliting the Data into Train, Test, and Validation sets

In [5]:
train_labels = label_list[label_list.index('1990-11-01'):label_list.index('2000-05-31')]
# train_labels = label_list[label_list.index('1983-11-01'):label_list.index('2016-05-31')]
# TODO: run above when using schooner
# should be what is above but you don't have enough space rn

test_labels = label_list[label_list.index('2016-11-01'):label_list.index('2019-05-31')]
val_labels = label_list[label_list.index('1979-11-01'):label_list.index('1982-05-31')]


In [12]:
train_labels = label_list[label_list.index('1990-11-01'):label_list.index('2000-05-31')]
test_labels = label_list[label_list.index('2016-11-01'):label_list.index('2019-05-31')]
val_labels = label_list[label_list.index('2013-11-01'):label_list.index('2016-05-31')]

In [10]:
train_era5_data = np.zeros((2119, 30, 51, 241, 2))


for i, val in enumerate(train_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].u250.values) == 0:
            train_era5_data[i,:,:, :,0] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].u250.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            train_era5_data[ i,:,:, :,0] = r
        else:
            d = get_data(val)[0].u250.values
            train_era5_data[i,:,:, :,0] = d
        # print(val)
    except Exception as e:
        train_era5_data[i,:,:, :,0] = np.full((30, 51, 241), 0)



for i, val in enumerate(train_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].z500.values) == 0:
            train_era5_data[i,:,:, :,1] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].z500.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            train_era5_data[i,:,:, :,1] = r
        else:
            d = get_data(val)[0].z500.values
            train_era5_data[i,:,:, :,1] = d
        # print(val)
    except Exception as e:
        train_era5_data[i,:,:, :,1] = np.full((30, 51, 241), 0)



train_olr_data = np.zeros(( 2119, 30, 13, 144, 1))
for i, val in enumerate(train_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[1].olr.values) == 0:
            train_olr_data[i,:,:, :,0] = np.full((30, 13, 144), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30,  13, 144))
            n = get_data(val)[1].olr.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 13, 144), 0)
            train_olr_data[i,:,:, :,0] = r
        else:
            d = get_data(val)[1].olr.values
            train_olr_data[i,:,:, :,0] = d
        # print(val)
    except Exception as e:
        train_olr_data[i,:,:, :,0] = np.full((30, 13, 144), 0)



In [11]:
train_pph_data = np.zeros((2119, 9))
for i, val in enumerate(train_labels):
    train_pph_data[i,:] = np.array(list(get_data(val)[2].to_dict().values()))


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.27907324, 0.08914221, ..., 0.        , 0.03325079,
        0.        ],
       [0.        , 0.08857632, 0.10432676, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.91484814, 0.        ,
        0.        ]])

In [12]:
test_era5_data = np.zeros((635, 30, 51, 241, 2))
for i, val in enumerate(test_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].u250.values) == 0:
            test_era5_data[i,:,:,:, 0] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].u250.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            test_era5_data[i,:,:,:, 0] = r
        else:
            d = get_data(val)[0].u250.values
            test_era5_data[i,:,:,:, 0] = d
        # print(val)
    except Exception as e:
        test_era5_data[i,:,:,:, 0] = np.full((30, 51, 241), 0)



for i, val in enumerate(test_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].z500.values) == 0:
            test_era5_data[i,:,:,:, 1] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].z500.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            test_era5_data[i,:,:,:, 1] = r
        else:
            d = get_data(val)[0].z500.values
            test_era5_data[i,:,:,:, 1] = d
        # print(val)
    except Exception as e:
        test_era5_data[i,:,:,:, 1] = np.full((30, 51, 241), 0)

test_olr_data = np.zeros((635, 30, 13, 144, 1))
for i, val in enumerate(test_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[1].olr.values) == 0:
            test_olr_data[i,:,:,:,0] = np.full((30, 13, 144), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30,  13, 144))
            n = get_data(val)[1].olr.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 13, 144), 0)
            test_olr_data[i,:,:,:,0] = r
        else:
            d = get_data(val)[1].olr.values
            test_olr_data[i,:,:,:,0] = d
        # print(val)
    except Exception as e:
        test_olr_data[i,:,:,:,0] = np.full((30, 13, 144), 0)


In [13]:
test_pph_data = np.zeros((635, 9))
for i, val in enumerate(test_labels):
    test_pph_data[i,:] = np.array(list(get_data(val)[2].to_dict().values()))


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.29205303e-04, 9.15009754e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.29787771e-02, 2.04527881e-03, 6.43886468e-02, ...,
        0.00000000e+00, 5.87768439e-01, 0.00000000e+00],
       [0.00000000e+00, 8.32211831e-06, 8.89534259e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.24768088e-02, 2.31149184e-01, 4.16156313e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [14]:
val_era5_data = np.zeros((635, 30, 51, 241, 2))

# print('here')
for i, val in enumerate(val_labels):
    # print(val)
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].u250.values) == 0:
            val_era5_data[i,:,:,:,0] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].u250.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            val_era5_data[i,:,:,:,0] = r
        else:
            d = get_data(val)[0].u250.values
            val_era5_data[i,:,:,:,0] = d
        # print(val)
    except Exception as e:
        val_era5_data[i,:,:,:,0] = np.full((30, 51, 241), 0)




for i, val in enumerate(val_labels):
    # print(val)
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[0].z500.values) == 0:
            val_era5_data[i,:,:,:,1] = np.full((30, 51, 241), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30, 51, 241))
            n = get_data(val)[0].z500.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 51, 241), 0)
            val_era5_data[i,:,:,:,1] = r
        else:
            d = get_data(val)[0].z500.values
            val_era5_data[i,:,:,:,1] = d
        # print(val)
    except Exception as e:
        val_era5_data[i,:,:,:,1] = np.full((30, 51, 241), 0)



val_olr_data = np.zeros((635, 30, 13, 144, 1))
for i, val in enumerate(val_labels):
    try:
        day_index = label_list.index(val)
        m = int(val.split('-')[1])
        d = int(val.split('-')[2])
        if len(get_data(val)[1].olr.values) == 0:
            val_olr_data[i,:,:,:,0] = np.full((30, 13, 144), 0)
        
        elif(m ==11) or (m ==12 and d < 7):
            r = np.zeros((30,  13, 144))
            n = get_data(val)[1].olr.values
            r[0:len(n), :, :] = n
            r[len(n):, :, :] = np.full(((30-len(n)), 13, 144), 0)
            val_olr_data[i,:,:,:,0] = r
        else:
            d = get_data(val)[1].olr.values
            val_olr_data[i,:,:,:,0] = d
        # print(val)
    except Exception as e:
        val_olr_data[i,:,:,:,0] = np.full((30, 13, 144), 0)

In [15]:
val_pph_data = np.zeros((635, 9))
for i, val in enumerate(val_labels):
    val_pph_data[i,:] = np.array(list(get_data(val)[2].to_dict().values()))


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 4.17033176e-03, 1.71284185e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.83343854e-02, 0.00000000e+00, 1.22424421e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.27130537e-04, 0.00000000e+00, 1.59175823e-03, ...,
        0.00000000e+00, 5.87768439e-01, 0.00000000e+00]])

### Model Building

Input(shape=(#steps,#features))

In [16]:
# creates a conv2d and max pool sequence 
# rounds: how many conv2d-max pool sequences wanted
# n_c2d_filters: filters
# num filters/rounds = how many conv2d layers will be in each round/sequence

def conv_2d_max_pool_seq (input_x, rounds, n_c2d_filters, activation, kernel_size=3):
  if len(n_c2d_filters)%rounds != 0:
    return "Incorrect number of filters"
  else:
    i = 0
    x = input_x
    d = int(len(n_c2d_filters)/rounds)
    while i < rounds:
      for n in range(d):
        x = layers.Conv3D(filters=n_c2d_filters[int(n+d*i)], kernel_size= kernel_size, activation=activation, padding='same')(x)
      # print(d)
      x = layers.MaxPooling3D((2,2,2))(x)
      i = i+1
    x = layers.Flatten()(x)
    return(x)
  

In [23]:
# TODO: make her better for hyperparameter tuning

input_wg = keras.Input(shape = ((30, 51, 241, 2)), name ="era5 input")
input_olr = keras.Input(shape = ((30, 13, 144, 1)), name ="olr input")

#dense output = 3 -> east, central, west 

#This model puts everything together; TODO: re-order parameters
def model_maker(input_wg, input_olr, rounds, n_c2d_filters, hidden_nodes, activation, name, dropout_rate, kernel_reg, kernel_size=3, learning_rate = 0.01,): 
    x = layers.concatenate([conv_2d_max_pool_seq(input_wg, rounds, n_c2d_filters, activation, kernel_size), 
                            conv_2d_max_pool_seq(input_olr, rounds, n_c2d_filters, activation, kernel_size)])
    
    #need to also concatenate 

    # for h in hidden_nodes:
    #     # x = layers.Dropout(dropout_rate)(x)
    #     x = layers.Dense(h, activation=activation)(x)

    outputs = layers.Dense(9)(x) #ridge regression for regulizer

    model = keras.Model(
        inputs = [input_wg, input_olr], 
        outputs = outputs, 
        name=name)
    

    opt = keras.optimizers.Adam(learning_rate = learning_rate,
                                   amsgrad = False)
    
    # Bind the model to the optimizer
    model.compile(loss='mse',
                  optimizer=opt,
                  metrics=['categorical_accuracy'])
    

    return model



In [24]:
model = model_maker(input_wg, input_olr, rounds=1, n_c2d_filters=[9], hidden_nodes=[9], 
            dropout_rate=0.001, activation='relu', kernel_reg='l1', name='first_iter')



In [1]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                      min_delta=0.01)

h = model.fit(x=[train_era5_data, train_olr_data], y=[train_pph_data], epochs=10, verbose=1,
                        validation_data=([val_era5_data, val_olr_data], [val_pph_data]), 
                        steps_per_epoch = 20)

NameError: name 'keras' is not defined